In [1]:
## Packages

# ##Installs
!pip install openpyxl transformers

##Imports
import pandas as pd
import transformers
import openpyxl

from sklearn.feature_extraction.text import TfidfVectorizer

c:\Users\julev\anaconda3\envs\python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## Reading in the PSYTar data set and parsing it

## Use sentence_labelling sheet

fileName=".\ONLINE_FORA\PsyTAR_dataset.xlsx"
data=pd.ExcelFile(fileName)
sheets={}
for sheet in data.sheet_names:
    sheets[sheet]=data.parse(sheet)

## Remove the first two sheets (License and read_me)
sheets.pop('License',None)
sheets.pop('read_me',None)

## This will print out the sheet names for the whole excel
for sheet in sheets.keys():
    print(f"Sheet_Name: {sheet}")

#To access a sheet, perform sheet['Sheet_Name']; e.g., sheets['Sample']

Sheet_Name: Sample
Sheet_Name: Sentence_Labeling
Sheet_Name: ADR_Identified
Sheet_Name: ADR_Mapped
Sheet_Name: WD_Identified
Sheet_Name: WD-Mapped 
Sheet_Name: SSI_Identified
Sheet_Name: SSI_Mapped
Sheet_Name: DI_Identified
Sheet_Name: DI_Mapped


In [9]:
## Vectorize data into TF-IDF

sheets['Sentence_Labeling'].fillna(0,inplace=True)

print(sheets['Sentence_Labeling'])

          id  comment_id        drug_id  sentence_index  \
0        1.0         1.0      lexapro.1             1.0   
1        2.0         1.0      lexapro.1             2.0   
2        3.0         1.0      lexapro.1             3.0   
3        4.0         1.0      lexapro.1             4.0   
4        5.0         1.0      lexapro.1             5.0   
...      ...         ...            ...             ...   
6005  1546.0       228.0  effexorXR.228            15.0   
6006  1547.0       228.0  effexorXR.228            16.0   
6007  1548.0       228.0  effexorXR.228            17.0   
6008  1549.0       228.0  effexorXR.228            18.0   
6009     0.0         0.0              0             0.0   

                                              sentences     ADR     WD  \
0     extreme weight gain, short-term memory loss, h...     1.0    0.0   
1                       I am detoxing from Lexapro now.     0.0    0.0   
2     I slowly cut my dosage over several months and...     0.0    0.

In [4]:
## Investigate K-means clustering to see whether certain medicines have different side effects - use as baseline to see if there is a significant difference with the transformer model


In [5]:
## Create a transformer

In [6]:
## Testing some changes